In [55]:
from functools import partial

import torch
import torch.nn as nn

# 画像系の復習はここに

In [46]:
# モデルの定義
conv_model = nn.Sequential(
    # [3, 28, 28]
    nn.Conv2d(1, 64, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    # [64, 14, 14]
    nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    # [128, 7, 7]
    nn.Conv2d(128, 64, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    # [64, 4, 4]
    # GAP
    nn.AdaptiveAvgPool2d(1),
    # [64, 1, 1]
    nn.Flatten(),
    # [64, 1]
    nn.Linear(64, 10)
   
)

In [47]:
X =  torch.randn(8, 1, 3, 3)
output = conv_model(X)
output.shape

torch.Size([8, 10])

In [49]:
# GAP層　動き確認
X =  torch.randn(8, 64, 4, 4)
X = nn.AdaptiveAvgPool2d(1)(X)
print(X.shape)

out = nn.Flatten()(X) # [8, 64]
out.shape

torch.Size([8, 64, 1, 1])


torch.Size([8, 64])

### hook

In [44]:
a = torch.ones(5, requires_grad=True)
b = 2 * a
b.retain_grad() # 中間層の勾配
b.register_hook(lambda grad : print(grad))
c = b.mean()
c.backward()

tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])


### nn.forward_hook

In [58]:
outputs = {}
def save_output(module, inp, out, name):
    module_name = f"{name}_{str(module)}"
    outputs[module_name] = out.shape

for name, module in conv_model.named_modules():
    if name:
        module.register_forward_hook(partial(save_output, name=name))